In [ ]:
import mbuild as mb
from foyer import Forcefield

In [ ]:
water = mb.load('files/tip3p.mol2')
water.name = 'SOL'

In [ ]:
c_ff = Forcefield('files/carbon.xml')
spce = Forcefield('files/spce.xml')

In [ ]:
system = mb.recipes.GraphenePoreSolvent(pore_depth=2,side_dim=2,
                    n_sheets=3,pore_width=1.2,x_bulk=1.5,
                    solvent=water,n_solvent=300)
box = mb.Box(system.periodicity)

In [ ]:
water = mb.Compound()
graphene = mb.Compound()
for child in system.children:
    if child.name == 'SOL':
        water.add(mb.clone(child))
    else:
        graphene.add(mb.clone(child))

water_pmd = spce.apply(water, residues='SOL')           
pore_pmd = c_ff.apply(graphene)

In [ ]:
system = water_pmd + pore_pmd
system.box[:3] = box.maxs * 10.0
system.save('init.gro', overwrite=True, combine='all')
system.save('init.top', overwrite=True, combine='all')

In [ ]:
!gmx grompp -f files/nvt.mdp -c init.gro -p init.top -o nvt.tpr

In [ ]:
!gmx mdrun -v -deffnm nvt